In [1]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time
import obspy
import numpy as np

# 0. 버전 확인 (1.4.2가 나와야 함)
print(f"--- 현재 obspy 버전: {obspy.__version__} ---")

client = Client("IRIS")

# --- 1. 2024년도 지진 '검색' ---
# (ID를 모르므로, 2024년 1월 1일 일본 노토 반도 지진(M7.5)을 검색)
print("--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---")
search_start = UTCDateTime("2024-01-01T00:00:00")
search_end = UTCDateTime("2024-01-02T00:00:00") # 1월 1일 하루 동안

origin_time = None
event_lat = None
event_lon = None
event_id_str = "N/A"

try:
    cat = client.get_events(
        starttime=search_start,
        endtime=search_end,
        minmagnitude=3.0 # M7.0 이상만 검색
    )

    if not cat:
        print("  > 2024년 1월 1일에 M3.0+ 지진을 찾을 수 없습니다.")
        print("  > 다른 날짜로 시도해보세요.")
        exit()

    # 검색된 첫 번째 이벤트(가장 큰 이벤트)를 사용
    event = cat[0]

    # 파형 수집에 필요한 정보 추출
    origin_time = event.origins[0].time
    event_lat = event.origins[0].latitude
    event_lon = event.origins[0].longitude
    event_mag = event.magnitudes[0].mag
    # event.resource_id는 객체일 수 있으므로 .id 속성 확인
    event_id_str = event.resource_id.id if event.resource_id else "ID_NOT_FOUND"

    print(f"  > 검색된 이벤트: {event.event_descriptions[0].text}")
    print(f"  > Event ID: {event_id_str}")
    print(f"  > 발생 시각 (UTC): {origin_time}")
    print(f"  > 규모: {event_mag}")
    print(f"  > 위치: {event_lat}, {event_lon}")

except Exception as e:
    print(f"  > get_events() 오류 (이벤트 검색 실패): {e}")
    exit()

# --- 2. '파형(Waveform)' 데이터 수집 ---
# (1단계에서 찾은 시간, 위치를 기준으로 파형을 요청)

print(f"\n--- 2. '{event_id_str}'의 파형(Waveform) 데이터 수집 시도... ---")
print("(조건: 발생 후 10분간, 진앙 5도 이내, BH? 채널)")

st = None # 파형 묶음 (Stream)

try:
    st = client.get_waveforms(
        network="*",        # 모든 네트워크
        station="*",      # 모든 관측소
        location="*",     # 모든 위치 코드
        channel="BH?",      # 광대역 채널 (BHZ, BHN, BHE)
        starttime=origin_time,              # 1. 검색된 지진 발생 시각
        endtime=origin_time + 600,          # 10분(600초) 동안
        latitude=event_lat,                 # 2. 검색된 위도
        longitude=event_lon,                # 3. 검색된 경도
        maxradius=5.0                       # 반경 5도 이내
    )

    if st:
        print(f"\n--- 3. 파형 수집 성공! ---")
        print(f"  > 총 {len(st)} 개의 파형(채널)을 다운로드했습니다.")

        # --- 4. "수치화된 데이터" (Number Array) 확인 ---
        print("\n--- 4. '수치화된 데이터' (숫자 배열) 확인 ---")

        tr = st[0] # 첫 번째 파형을 샘플로 확인
        print(f"  > 첫 번째 파형 ID: {tr.id}")

        # <<< 이것이 바로 모델에 넣을 "숫자 배열"입니다 >>>
        number_array = tr.data

        print(f"  > 이 파형의 총 숫자 개수: {len(number_array)} 개")
        print(f"  > 실제 숫자 배열 (앞 10개): {number_array[:10]}")

        # --- 5. 특징(Feature) 계산 예시 ---
        print("\n--- 5. 이 '숫자 배열'로 특징(Feature) 계산하기 (예시) ---")

        max_amplitude = np.max(np.abs(number_array))

        print(f"  > (예시 특징) 최대 진폭 (Max Amplitude): {max_amplitude:.2f}")

    else:
        print(f"\n--- 3. 수집 실패 (데이터 없음) ---")
        print("  > 이 조건(2024년 1월 1일)에 맞는 파형 데이터가 서버에 없습니다.")

except AttributeError as ae:
    print(f"\n--- !!! 심각한 오류 발생 (AttributeError) !!! ---")
    print(f"  > 오류: {ae}")
    print(f"  > 원인: 'obspy.Client'가 'get_waveforms' 함수도 찾지 못합니다.")
    print(f"  > 해결: Python 3.11 가상환경 폴더(venv)를 삭제하고 2단계부터 다시 시도해야 합니다.")

except Exception as e:
    print(f"\n--- !!! 오류 발생 !!! ---")
    print(f"  > 오류: {e}")
    if "No data available" in str(e) or "204" in str(e):
        print("  > (원인: 이 조건에 맞는 데이터가 서버에 없습니다.)")
    else:
        print("  > (원인: 그 외 오류. 네트워크 문제일 수 있습니다.)")

--- 현재 obspy 버전: 1.4.2 ---
--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---
  > 검색된 이벤트: SOUTH OF ALASKA
  > Event ID: smi:service.iris.edu/fdsnws/event/1/query?eventid=11792788
  > 발생 시각 (UTC): 2024-01-01T23:27:32.613000Z
  > 규모: 3.2
  > 위치: 54.3122, -157.3843

--- 2. 'smi:service.iris.edu/fdsnws/event/1/query?eventid=11792788'의 파형(Waveform) 데이터 수집 시도... ---
(조건: 발생 후 10분간, 진앙 5도 이내, BH? 채널)

--- !!! 오류 발생 !!! ---
  > 오류: The parameter 'latitude' is not supported by the service.
  > (원인: 그 외 오류. 네트워크 문제일 수 있습니다.)


In [2]:
import pandas as pd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import time
import obspy
import numpy as np
import os

print(f"--- 현재 obspy 버전: {obspy.__version__} ---")
client = Client("IRIS")

# --- 1. 2024년도 지진 '검색' ---
print("--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---")
search_start = UTCDateTime("2024-02-01T00:00:00")
search_end = UTCDateTime("2024-03-01T00:00:00")
min_mag = 3.0

results_list = []

try:
    cat = client.get_events(
        starttime=search_start,
        endtime=search_end,
        minmagnitude=min_mag
    )
    if not cat:
        print(f"  > {search_start.date}에 M{min_mag}+ 지진을 찾을 수 없습니다.")
        exit()

    print(f"  > 총 {len(cat)} 개의 M{min_mag}+ 지진을 찾았습니다. 파형 수집을 시작합니다.")

    # --- 2. '모든' 지진을 순회 ---
    for i, event in enumerate(cat):

        event_time = None
        event_lat = None
        event_lon = None
        event_mag = None
        event_depth_km = None
        event_id_str = "N/A"

        # 검색할 반경 (단위: 도)
        search_radius_deg = 2.0

        try:
            # --- 3. 기본 지진 정보 추출 ---
            origin = event.origins[0]
            event_time = origin.time
            event_lat = origin.latitude
            event_lon = origin.longitude
            event_depth_km = origin.depth / 1000.0 if origin.depth else 0.0
            event_mag = event.magnitudes[0].mag if event.magnitudes else 0.0

            if event.resource_id:
                event_id_str = str(event.resource_id.id).split('=')[-1]
            else:
                event_id_str = f"event_{i}"

            print(f"\n--- [{i+1}/{len(cat)}] 이벤트 처리 중 (ID: {event_id_str}, Mag: {event_mag}) ---")

            # --- 4. (!!! 여기가 수정된 부분 1 !!!) ---
            # 먼저 지진 근처의 '관측소(Station)' 목록을 가져옵니다.
            print(f"  > 4a. 반경 {search_radius_deg}도 이내의 관측소 검색 중...")

            stations_inventory = client.get_stations(
                starttime=event_time,
                endtime=event_time + 300, # 5분
                latitude=event_lat,       # get_stations는 이 파라미터가 작동해야 합니다.
                longitude=event_lon,
                maxradius=search_radius_deg,
                level="station"
            )

            if not stations_inventory or len(stations_inventory) == 0:
                print("  > 근처에 관측소를 찾지 못했습니다. (데이터 없음). 다음 이벤트로 넘어갑니다.")
                continue

            # 관측소 목록에서 '네트워크'와 '관측소' 코드만 추출합니다.
            network_codes = set()
            station_codes = set()
            for network in stations_inventory:
                network_codes.add(network.code)
                for station in network:
                    station_codes.add(station.code)

            if not station_codes:
                 print("  > 관측소 코드를 추출하지 못했습니다. 다음 이벤트로 넘어갑니다.")
                 continue

            # 콤마(,)로 구분된 문자열로 변환 (e.g., "JP,KR", "HON,INU")
            net_str = ",".join(network_codes)
            sta_str = ",".join(station_codes)

            print(f"  > 4b. 찾은 관측소({len(station_codes)}개)의 파형 요청 중...")

            # --- 5. (!!! 여기가 수정된 부분 2 !!!) ---
            # 'minlatitude' 대신 'network', 'station' 코드로 파형 요청
            st = client.get_waveforms(
                network=net_str,    # (수정) 네트워크 코드로 필터
                station=sta_str,    # (수정) 관측소 코드로 필터
                location="*",
                channel="BHZ",
                starttime=event_time,
                endtime=event_time + 300
                # (삭제) minlatitude, maxlatitude, minlongitude, maxlongitude
            )

            max_amplitude = 0.0

            if st:
                max_amps_per_station = [np.max(np.abs(tr.data)) for tr in st if len(tr.data) > 0]
                if max_amps_per_station:
                    max_amplitude = np.max(max_amps_per_station)
                print(f"  > 파형 수집 성공. 최대 진폭 (Max Amp): {max_amplitude:.2f}")
            else:
                print(f"  > 이 이벤트의 파형 데이터를 찾을 수 없습니다 (No data available).")

            # --- 6. CSV 저장을 위해 리스트에 '결과' 추가 ---
            results_list.append({
                "event_id": event_id_str,
                "time_utc": event_time.isoformat(),
                "latitude": event_lat,
                "longitude": event_lon,
                "magnitude": event_mag,
                "depth_km": event_depth_km,
                "max_amplitude_bhz": max_amplitude
            })

        except Exception as e:
            # 만약 get_stations()에서도 'latitude' 오류가 난다면, obspy.Client 자체가 완전히 고장 난 것입니다.
            print(f"  > 이벤트 처리 중 치명적 오류 발생: {e}")
            continue

except Exception as e:
    print(f"  > get_events() 오류 (이벤트 검색 실패): {e}")
    exit()

# --- 7. 모든 작업 완료 후, 리스트를 Pandas DataFrame으로 변환 ---
print("\n--- 7. 모든 이벤트 처리 완료. CSV 파일로 저장합니다. ---")

if results_list:
    df = pd.DataFrame(results_list)
    output_filename = f"earthquake_features_{search_start.date}.csv"
    output_path = os.path.join(os.getcwd(), output_filename)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')

    print(f"  > 성공! 총 {len(df)} 개의 이벤트 데이터를")
    print(f"  > '{output_path}' 파일로 저장했습니다.")
    print("\n[CSV 샘플 데이터]")
    print(df.head())
else:
    print("  > 저장할 데이터가 없습니다. (모든 이벤트에서 파형 수집 실패)")

print("\n--- 작업 종료 ---")

--- 현재 obspy 버전: 1.4.2 ---
--- 1. 2024년 1월 1일 (M3.0+) 지진 검색 중... ---
  > 총 1439 개의 M3.0+ 지진을 찾았습니다. 파형 수집을 시작합니다.

--- [1/1439] 이벤트 처리 중 (ID: 11814988, Mag: 4.2) ---
  > 4a. 반경 2.0도 이내의 관측소 검색 중...
  > 4b. 찾은 관측소(3개)의 파형 요청 중...
  > 파형 수집 성공. 최대 진폭 (Max Amp): 32622.00

--- [2/1439] 이벤트 처리 중 (ID: 11811116, Mag: 4.1) ---
  > 4a. 반경 2.0도 이내의 관측소 검색 중...
  > 이벤트 처리 중 치명적 오류 발생: No data available for request.
HTTP Status code: 204
Detailed response of server:



--- [3/1439] 이벤트 처리 중 (ID: 11814983, Mag: 4.4) ---
  > 4a. 반경 2.0도 이내의 관측소 검색 중...
  > 이벤트 처리 중 치명적 오류 발생: No data available for request.
HTTP Status code: 204
Detailed response of server:



--- [4/1439] 이벤트 처리 중 (ID: 11814980, Mag: 4.0) ---
  > 4a. 반경 2.0도 이내의 관측소 검색 중...
  > 이벤트 처리 중 치명적 오류 발생: No data available for request.
HTTP Status code: 204
Detailed response of server:



--- [5/1439] 이벤트 처리 중 (ID: 11811058, Mag: 3.2) ---
  > 4a. 반경 2.0도 이내의 관측소 검색 중...
  > 4b. 찾은 관측소(121개)의 파형 요청 중...
  > 파형 수집 성공. 최대 진폭 (Max Amp): 94594.0